# 패키지, 데이터 로드

In [2]:
# 패키지 로드 
import pandas as pd 
from bs4 import BeautifulSoup  # html 파싱 
import re  # 정규표현식

In [3]:
# 데이터 로드 
file_path = '../../../data/mirae/2023BigFesta_NewsDataset_202303_5.xlsx'  
news = pd.read_excel(file_path)
news.head() # 확인 

,Unnamed: 0,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,3072,202303,20230313_130339,로이터(AI뉴스),정치,"러시아 방공, 벨고로드 지역 미사일 4기 격추 - 주지사",16.57,NaN,우크라이나 |소재 |이머징마켓 |아시아 |EU |러시아 |미국 |,<html>\n<head>\n<style>\n\n * {\n ...
1,3073,202303,20230313_134543,로이터(AI뉴스),기술,미국의 SVB 붕괴 개입 이후 주요 암호주화 안정,16.41,|라이엇 플랫폼스|SVB 파이낸셜 그룹|매러선 디지털 홀딩스|코인베이스 글로벌|,금융 |미국 |,<html>\n<head>\n<style>\n\n * {\n ...
2,3074,202303,20230313_140702,로이터(AI뉴스),경제,"신흥 시장 - 태국 바트화, 미 달러화 약세로 아시아 통화 상승",63.53,SVB 파이낸셜 그룹|,중국 |금융 |베트남 |아시아 |헬스케어 |인도네시아 |화학 |미국 |대한민국 |상...,<html>\n<head>\n<style>\n\n * {\n ...
3,3075,202303,20230313_152734,로이터(AI뉴스),경제,"미국 S&P 500 E-MINI 선물 1.79%, 나스닥 선물 1.87% 상승",63.50,NaN,다우 |S&P; |미국 |나스닥 |선물,<html>\n<head>\n<style>\n\n * {\n ...
4,3076,202303,20230313_150001,로이터(AI뉴스),경제,도쿄도 닛케이평균주가 1.11% 하락,57.77,NaN,일 본 |아시아 |,<html>\n<head>\n<style>\n\n * {\n ...


In [4]:
news.drop('Unnamed: 0',axis = 1, inplace = True) # 불필요한 열 삭제 
news.head() # 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,202303,20230313_130339,로이터(AI뉴스),정치,"러시아 방공, 벨고로드 지역 미사일 4기 격추 - 주지사",16.57,NaN,우크라이나 |소재 |이머징마켓 |아시아 |EU |러시아 |미국 |,<html>\n<head>\n<style>\n\n * {\n ...
1,202303,20230313_134543,로이터(AI뉴스),기술,미국의 SVB 붕괴 개입 이후 주요 암호주화 안정,16.41,|라이엇 플랫폼스|SVB 파이낸셜 그룹|매러선 디지털 홀딩스|코인베이스 글로벌|,금융 |미국 |,<html>\n<head>\n<style>\n\n * {\n ...
2,202303,20230313_140702,로이터(AI뉴스),경제,"신흥 시장 - 태국 바트화, 미 달러화 약세로 아시아 통화 상승",63.53,SVB 파이낸셜 그룹|,중국 |금융 |베트남 |아시아 |헬스케어 |인도네시아 |화학 |미국 |대한민국 |상...,<html>\n<head>\n<style>\n\n * {\n ...
3,202303,20230313_152734,로이터(AI뉴스),경제,"미국 S&P 500 E-MINI 선물 1.79%, 나스닥 선물 1.87% 상승",63.50,NaN,다우 |S&P; |미국 |나스닥 |선물,<html>\n<head>\n<style>\n\n * {\n ...
4,202303,20230313_150001,로이터(AI뉴스),경제,도쿄도 닛케이평균주가 1.11% 하락,57.77,NaN,일 본 |아시아 |,<html>\n<head>\n<style>\n\n * {\n ...


# 키워드 지정
- 아래의 키워드를 뉴스 본문 html 이 저장된 변수인 'CONTENT'변수에 2번 이상 포함하지 않을 경우 엔터테인먼트와 무관한 기사로 판단하여 제거한다.

In [9]:
keywords = [
    '방시혁','하이브','에스엠','이수만','와이지','양현석','제이와이피','박진영',
    
    '뉴진스','방탄소년단','르세라핌',
    '세븐틴','투모로우바이투게더',
    '프로미스나인','나나','이현','미드낫','엔하이픈',
    '백호','강동호','황민현','예하나','성연','지코',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    '블랙핑크','트레저','악뮤','악동뮤지션',
    '위너','지디','권지용',
    '젝스키스',
    
    '트와이스','엔믹스', '데이식스',
    'itzy','채령','류진','유나','예지'
    
    '스트레이키즈','투피엠','보이스토리',
    '야오천','니지유','엑스디너리 히어로즈',
    'boa',
    '에스파',
    '엑소',
    
    '샤이니','엔시티','엔시티 127','엔시티 드림',    
    '동방신기','슈퍼주니어','소녀시대','엑소'
    '레드벨벳','웨이션브이','슈퍼엠',
    '갓더비트','갓 더 비트','라이즈'

]

# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)

# 'PARSED CONTENT' 열에서 정규식 패턴을 검색하여 추출
filtered_df = news[news['CONTENT'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)]

# 특정 키워드가 2번 이상 포함된 행만 남기기
filtered_df = filtered_df[filtered_df['CONTENT'].apply(lambda text: sum(1 for keyword in keywords if keyword.lower() in text.lower()) >= 2)]


In [10]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3336 entries, 166 to 273869
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NEWS_YM      3336 non-null   int64  
 1   DATE_TIME    3336 non-null   object 
 2   WRITER       3336 non-null   object 
 3   SUBCATEGORY  3336 non-null   object 
 4   TITLE        3336 non-null   object 
 5   IMPORTANCE   3336 non-null   float64
 6   ITEM_NAME    1458 non-null   object 
 7   TAG_LIST     1951 non-null   object 
 8   CONTENT      3336 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 260.6+ KB


# 데이터 저장

In [11]:
# 데이터 저장
file_path = '../../../data/News/news_1filtered_202303.xlsx'  
filtered_df.to_excel(file_path, index=False)